# M1N Folkersma, pt 1

**Data source:** email from Folkersma

**Processing objective:** obtain a (routable!), fully connected network object for each of the 3 regions in the data set

**Processing steps:**

Workflow to create a network from Folkersma data.
* **Step 1 (Python): from raw data to QGIS input**
* **Step 2 (QGIS): from QGIS input to nx input**
* Step 3 (Python): from nx input to graph object

## Step 1

In [1]:
# Load libraries
import numpy as np
import os
# import pandas as pd
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
# import shapely
# from shapely.geometry import Point, LineString, MultiLineString
# from collections import Counter
# import matplotlib.pyplot as plt
# import networkx as nx

**0. Define proj crs!!**

In [2]:
proj_crs = "EPSG:25832"

**1. Read in data**

In [3]:
gdf = gpd.read_file("../../data/folkersma/raw/stretch.shp")

**2. Check CRS and convert to projected**

In [4]:
# assert gdf.crs is not None, "crs must be set!"
gdf = gdf.set_crs("EPSG:4326")
gdf = gdf.to_crs(proj_crs)

**3. Check data set features**

In [5]:
gdf.head(3)

,id,UUID,status,rating,region,stretch,private,oneway,roadsurfac,length,comment,geometry
0,1834,l1834,Concept,1.0,NÃ¦stved,00-00,0.0,1.0,0.0,0.029656,NaN,"LINESTRING (675365.070 6123560.480, 675365.580..."
1,1766,l1766,Concept,1.0,NÃ¦stved,00-00,0.0,1.0,0.0,1.528090,NaN,"LINESTRING (674260.732 6124551.179, 674266.410..."
2,1836,l1836,Concept,1.0,NÃ¦stved,00-00,0.0,1.0,0.0,0.048249,NaN,"LINESTRING (675370.180 6123535.010, 675367.890..."


In [6]:
### only columns to keep: rating and geometry
gdf = gdf[["rating", "geometry"]]

**4. Assert: one linestring geometry per row**

In [7]:
gdf = gdf.explode(index_parts = False).reset_index(drop=True)
assert all(gdf.geometry.type=="LineString")
assert all(gdf.geometry.is_valid)

**5. Save temporary file for QGIS processing**

In [8]:
gdf.to_file("../../data/folkersma/processed/qgis_input.gpkg", index = False)

***
***
## Step 2 (in QGIS)

**6. Add nodes at all intersections ("split with lines") and snap endpoints to each other**


In [9]:
from qgis.core import *
from PyQt5.QtCore import QVariant

In [10]:
myinputfile = "../../data/folkersma/processed/qgis_input.gpkg"
myoutputfile = "../../data/folkersma/processed/qgis_output.gpkg"

In [11]:
# SETTINGS FOR SNAPPING
mytolerance = 5
mybehaviour = 6

In [12]:
# output of running `QgsApplication.prefixPath()` in the Python Console of QGIS
myprefix = '/Applications/QGIS-LTR.app/Contents/MacOS'

# Supply path to qgis install location
QgsApplication.setPrefixPath(
    myprefix, 
    False # use default paths
    )

# Create a reference to the QgsApplication. 
qgs = QgsApplication(
    [], 
    False # do *not* use GUI
    )

# Load providers
qgs.initQgis()

qt.qpa.fonts: Populating font family aliases took 242 ms. Replace uses of missing font family "Open Sans" with one that exists to avoid this cost. 


In [13]:
# import and initialize processing
import processing
from processing.core.Processing import Processing
Processing.initialize()

In [14]:
# Run processing algorithm "split with lines"
temp_out_split = processing.run(
   "native:splitwithlines",
       {
           'INPUT':myinputfile,
           'LINES':myinputfile,
           'OUTPUT':'TEMPORARY_OUTPUT'
       }
   )
print("done: split with lines")


done: split with lines


**Snap geometries**

In [15]:
# snap
temp_out_snap = processing.run(
    "native:snapgeometries",
        {
            'INPUT':temp_out_split["OUTPUT"],
            'REFERENCE_LAYER':temp_out_split["OUTPUT"],
            'TOLERANCE':mytolerance,
            'BEHAVIOR':mybehaviour,
            'OUTPUT':'TEMPORARY_OUTPUT'
         }
    )
print(f"done: snapped with tolerance {mytolerance}, behaviour {mybehaviour}")


done: snapped with tolerance 5, behaviour 6


**Check validity**

In [16]:
temp_out_validity = processing.run(
    "qgis:checkvalidity",
        {
            'INPUT_LAYER': temp_out_snap["OUTPUT"],
            'METHOD': 2,
            'IGNORE_RING_SELF_INTERSECTION': False,
            'VALID_OUTPUT': 'TEMPORARY_OUTPUT',
            'INVALID_OUTPUT':None,
            'ERROR_OUTPUT':None
        }
    )
print("done: validity check")

done: validity check


**Delete linestrings of just 1 point**

In [17]:
vlayer = temp_out_validity["VALID_OUTPUT"]
layer_provider=vlayer.dataProvider()

# add a "mylength" colum to the attribute table
layer_provider.addAttributes([QgsField("mylength",QVariant.Double)])
vlayer.updateFields()

# fill "mylength" column with length values
vlayer.startEditing()
for f in vlayer.getFeatures():
    id=f.id()
    length=f.geometry().length()
    attr_value={2:length}
    layer_provider.changeAttributeValues({id:attr_value})
vlayer.commitChanges()

# find strings with length 0
expression = 'mylength = 0'
request = QgsFeatureRequest().setFilterExpression(expression)
matches = []
for f in vlayer.getFeatures(request):
   matches.append(f["fid"])

# erase length 0 strings
if vlayer.dataProvider().capabilities() & QgsVectorDataProvider.DeleteFeatures:
    print("layer supports deletion")
    res = vlayer.dataProvider().deleteFeatures(matches)

# delete "mylength" field
vlayer.dataProvider().deleteAttributes([0, 2])

print("done: delete linestrings with length 0")


layer supports deletion
done: delete linestrings with length 0


**Export results**

In [18]:
processing.run(
    "native:package",
        {
            'LAYERS': vlayer,
            'OUTPUT': myoutputfile,
            'OVERWRITE':True,
            'SAVE_STYLES':False,
            'SAVE_METADATA':True,
            'SELECTED_FEATURES_ONLY':False,
            'EXPORT_RELATED_LAYERS':False
        }
    )

{'OUTPUT': '../../data/folkersma/processed/qgis_output.gpkg',
 'OUTPUT_LAYERS': ['../../data/folkersma/processed/qgis_output.gpkg|layername=Valid output']}

In [19]:
qgs.exitQgis()